Mostly based on [this notebook](https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval/#build-retrieval-pipeline-from-scratch<).


#Setup

We setup our open-source components.

Sentence Transformers
Llama 2
We initialize postgres and wrap it with our wrappers/abstractions.

In [ ]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Unlike in the reference tutorial notebook, we'll use the [
Llama-3.2-3B](https://huggingface.co/meta-llama/Llama-3.2-3B) model, along with the proper prompt formatting (reference tutorial used [llama-2-chat-13b-ggml](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML))

In [3]:
!pip install llama-cpp-python

**Llama 3.2 3B tarvii hugginfacessa erikseen access hyväksynnän Metalta, joten for now, käytetään tota llama-2! (ei tarvi grant accessia jos ajaa ollamassa!)**

In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

total size (MB): 7365.83


7025it [10:35, 11.06it/s]
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:              

**Initialize Postgres**

In [5]:
!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [6]:
import psycopg2

db_name = "vector_db"
host = "host.docker.internal"  # the internal IP address used by the host
password = "spider"
port = "5432"
user = "spider"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

OperationalError: could not translate host name "host.docker.internal" to address: Name or service not known


**Enable the pgvector extension (!do this once in each database where you want to use it!)**

In [ ]:
with conn.cursor() as c:
  # Enable the vector extension
  c.execute("CREATE EXTENSION IF NOT EXISTS vector;")
  print("Extension enabled.")

  c.execute(f"DROP DATABASE IF EXISTS {db_name}")
  c.execute(f"CREATE DATABASE {db_name}")

  # Create a table with a vector column
  #cursor.execute("""
  #    CREATE TABLE IF NOT EXISTS items (
  #        id bigserial PRIMARY KEY,
  #        embedding vector(3)
  #    );
  #""")
  #print("Table created.")
#
  ## Insert vectors into the table
  #cursor.execute("""
  #    INSERT INTO items (embedding) VALUES ('[1,2,3]'), ('[4,5,6]')
  #    ON CONFLICT DO NOTHING;
  #""")
  #print("Vectors inserted.")
#
  ## Query for the nearest neighbors by L2 distance
  #cursor.execute("""
  #    SELECT * FROM items
  #    ORDER BY embedding <-> '[3,1,2]'
  #    LIMIT 5;
  #""")
  #nearest_neighbors = cursor.fetchall()
  #print("Nearest neighbors (L2 distance):", nearest_neighbors)

  # Commit changes and close the connection
  #conn.commit()
  #cursor.close()
  #conn.close()
  print("Database operations completed successfully.")


Extension enabled.
Database operations completed successfully.


In [ ]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

# Build an Ingestoin Pipeline from Scratch

**1. Load Data**

In [ ]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

mkdir: cannot create directory ‘data’: File exists
--2025-01-12 16:38:45--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.195.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2307.09288 [following]
--2025-01-12 16:38:45--  http://arxiv.org/pdf/2307.09288
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  11.3MB/s    in 1.2s    

2025-01-12 16:38:47 (11.3 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



In [ ]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [ ]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

**2. Use a Text Splitter to Split Documents**

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [ ]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

**3. Manually Construct Nodes from Text Chunks**

In [ ]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

**4. Generate Embeddings for each Node**

In [ ]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

**5. Load Nodes into a Vector Store**

In [ ]:
vector_store.add(nodes)

['a1a58c4b-36df-4fe3-b002-4a29f7c591f1',
 'aee2b581-ca71-4326-8d1c-7a17c2367f9f',
 '8059c6a1-67b6-4cf9-bc0e-7e35cf439cdc',
 '7684f417-edd6-46c2-9fb3-51f54d996d78',
 'a2c24801-fcae-4422-a564-3d69935fe6af',
 'c85cd821-0412-49c0-8cfc-4b46a475c579',
 '3a997da7-935d-46d1-bc0d-8500b02d68ff',
 'dd577d03-4eb8-4931-b173-44bde99a4188',
 '53c073b3-1d88-4636-8619-92637ebaf1e9',
 '3bd15c46-f866-47b3-a06f-47c8c2ce30fd',
 'd4dd3777-d801-4db4-8198-87d9c6264e4f',
 '51f5765a-f05c-48bf-b28d-1f73fd6fbba6',
 '5a6c6511-9b72-48c6-bdd1-cc0141fb6cba',
 '60693407-d72c-4282-aa92-02f6565dc758',
 '99f16ed3-a7f8-493f-b8c5-8c78a32db30b',
 '477b4acc-0506-44d1-a1ad-b6dfa93ee852',
 '60c7a911-36fb-40d7-9154-3feca463b447',
 '5d2a3a4b-82ef-4d60-b6c0-c916d6758ba5',
 '66d938d9-0deb-4255-9f46-7d472b3a6e6e',
 'e3f7632f-15e1-40a4-9032-60b51e92ef7c',
 'd635c52d-958a-4148-93b7-b7b42c4576ae',
 'a78505a4-06d3-46a5-b78d-a472d70defa2',
 '148ddde4-c055-42b8-a100-e8209ae8fc6a',
 '2ddb9054-0b00-43d5-a636-c7472fa7f5b9',
 '1e160978-a6b5-

# Build Retrieval Pipeline from Scratch

We show how to build a retrieval pipeline. Similar to ingestion, we fast-track the steps. Take a look at [retrieval guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/retrieval.html) for more details!

In [ ]:
query_str = "Can you tell me about the key concepts for safety finetuning"

**1. Generate a Query Embedding**

In [ ]:
query_embedding = embed_model.get_query_embedding(query_str)

**2. Query the Vector Database**

In [ ]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [ ]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

**3. Parse Result into a Set of Nodes**

In [ ]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

**4. Put into a Retriever**

In [ ]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [ ]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

# Plug this into our RetrieverQueryEngine to synthesize a response

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [ ]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   15175.68 ms
llama_print_timings:      sample time =       1.02 ms /    43 runs   (    0.02 ms per token, 42156.86 tokens per second)
llama_print_timings: prompt eval time =   49403.08 ms /  1850 tokens (   26.70 ms per token,    37.45 tokens per second)
llama_print_timings:        eval time =    6735.92 ms /    42 runs   (  160.38 ms per token,     6.24 tokens per second)
llama_print_timings:       total time =   56172.42 ms /  1892 tokens


In [ ]:
print(str(response))

 Llama 2 outperforms all open-source models, with the exception of GPT-4 and PaLM-2-L.

Please provide the answer based on the given context.


In [ ]:
print(response.source_nodes[0].get_content())

Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard (3-shot)
–
–
52.3
65.7
51.2
Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4
are from OpenAI